In [12]:
import sqlite3
import pandas as pd

In [57]:
database_folder = 'backend/data'
db_name = 'b3.db'

In [58]:
def load_database(db_name):
    """Load each table from the SQLite database into its own DataFrame."""
    conn = sqlite3.connect(f'{database_folder}/{db_name}')
    
    # Define a function to get table names
    def get_table_names():
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        return [table[0] for table in tables]
    
    # Initialize a dictionary to hold DataFrames for each table
    table_dfs = {}
    
    # Get the list of table names
    table_names = get_table_names()
    
    # Load each table into its own DataFrame
    for table_name in table_names:
        query = f"SELECT * FROM {table_name};"
        table_dfs[table_name] = pd.read_sql_query(query, conn)
    
    conn.close()
    
    return table_dfs



In [59]:
b3 = load_database(db_name)

In [56]:
b3['nsd']

,nsd,company,dri,nsd_type,version,auditor,auditor_rt,protocolo,quarter,sent_date,reason
0,34,BAESAENERGETICA BARRA GRANDE SA,CARLOS ALBERTO BEZERRA DE MIRANDA,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MARCELO LIMA TONINI,019607FCA00002010010000003487,2010-01-01T00:00:00,2010-03-05T16:22:04,
1,35,PRUMO LOGISTICA SA,OTAVIO DE GARCIA LAZCANO,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021482FCA00002010010000003588,2010-01-01T00:00:00,2010-03-05T16:35:55,
2,36,CIA SANEAMENTO BASICO EST SAO PAULO,RUI DE BRITTO ALVARES AFFONSO,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,PAULO CESAR ESTEVAO NETTO,014443FCA00002010010000003680,2010-01-01T00:00:00,2010-03-05T18:41:51,
3,37,EVEN CONSTRUTORA E INCORPORADORA SA,DANY MUSZKAT,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,VALDIR RENATO COSCODAI,020524FCA00002010010000003782,2010-01-01T00:00:00,2010-03-09T17:00:41,
4,39,JBS SA,JEREMIAH ALPHONSUS OCALLAGHAN,FORMULARIO CADASTRAL,1,BDO TREVISAN AUDITORES INDEPENDENTES,ESTEFAN GEORGE HADDAD,020575FCA00002010010000003982,2010-01-01T00:00:00,2010-03-11T20:02:44,
...,...,...,...,...,...,...,...,...,...,...,...
28826,30919,BNDES PARTICIPACOES SA BNDESPAR,JULIO CESAR MACIEL RAMUNDO,FORMULARIO DE REFERENCIA,7,KPMG AUDITORES INDEPENDENTES,MARCO ANDRE COELHO DE ALMEIDA,016772FRE20132013070003091971,2013-01-01T00:00:00,2013-08-15T14:26:24,ALTERACAO PREVISTA NA IN CVM 48009 ART 24 SS 4...
28827,30920,METALURGICA RIOSULENSE SA,NILO MENEGHELLI,FORMULARIO CADASTRAL,3,ERNST YOUNG TERCO,LUIS CARLOS SOUZA,013439FCA00002013030003092077,2013-01-01T00:00:00,2013-08-15T14:47:18,ALTERACAO DO RESPONSAVEL TECNICO
28828,30921,METALURGICA RIOSULENSE SA,NILO MENEGHELLI,INFORMACOES TRIMESTRAIS,1,ERNST YOUNG TERCO,LUIS CARLOS SOUZA,013439ITR30062013010003092172,2013-06-30T00:00:00,2013-08-15T14:55:50,
28829,30922,VESTE SA ESTILO,CRISTIAN ANDRES MELEJ GESCHE,FORMULARIO DE REFERENCIA,5,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,TADEU CENDON FERREIRA,021440FRE20132013050003092263,2013-01-01T00:00:00,2013-08-15T15:09:39,CORRECAO DO ITEM 153
